In [2]:
import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import math
import os
import cv2
import json
import numpy as np
from pprint import pprint
from torch.autograd import Variable
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
from ipywidgets import FloatProgress
from IPython.display import display


%matplotlib inline

class VGG(nn.Module):

   def __init__(self, features, num_classes=2):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(512*2*2, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        self._initialize_weights()

   def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        #print(x.size())
        x = self.classifier(x)
        return F.softmax(x)

   def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

In [155]:
def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 1
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [156]:
def vgg16(pretrained=False, **kwargs):
    """VGG 16-layer model (configuration "D")

   Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = VGG(make_layers(cfg['D']), **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['vgg16']))
    return model

In [157]:
def vgg16_bn(pretrained=False, **kwargs):
    """VGG 16-layer model (configuration "D") with batch normalization

   Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = VGG(make_layers(cfg['D'], batch_norm=True), **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['vgg16_bn']))
    return model

In [158]:
vgg = vgg16_bn()
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(vgg.parameters(), lr=0.001, momentum=0.9)

In [159]:
# Create dataloader
DATA_COLLECTION_ROOT = "../data_collection/"
POS_SAMPLES = DATA_COLLECTION_ROOT + "pos_moles/cropped/"
NEG_SAMPLES = DATA_COLLECTION_ROOT + "neg_moles/cropped/"

print("Collecting samples...")
# Collect samples and sort them into positive and negative
pos_paths = []
neg_paths = []
for path in os.listdir(POS_SAMPLES):
    if path.endswith(".png"):
        pos_paths.append(POS_SAMPLES + path)

for path in os.listdir(NEG_SAMPLES):
    if path.endswith(".png"):
        neg_paths.append(NEG_SAMPLES + path)

# Limit sample sizes for quicker testing
pos_paths = pos_paths[:1500]
neg_paths = neg_paths[:1500]
        
sample_img_shape = cv2.imread(pos_paths[0]).shape
print("Image shape: %d x %d x %d" % sample_img_shape)
#sis_flat = cv2.imread(pos_paths[0]).flatten().shape

num_samples = 0

# to optimize memory, preallocate np arrays
for img_path in pos_paths:
    img = cv2.imread(img_path)
    if img.shape == sample_img_shape:
        num_samples += 1

for img_path in neg_paths:
    img = cv2.imread(img_path)
    if img.shape == sample_img_shape:
        num_samples += 1

x_size = (num_samples, sample_img_shape[0], sample_img_shape[1], sample_img_shape[2])
y_size = (num_samples, 2)

all_x = np.zeros(x_size, dtype=np.uint8)
all_y = np.zeros(y_size, dtype=np.uint8)

sample_i = 0
for img_path in pos_paths:
    img = cv2.imread(img_path)
    if img.shape == sample_img_shape:
        all_x[sample_i] = img
        all_y[sample_i] = np.array([0,1])
        sample_i += 1

for img_path in neg_paths:
    img = cv2.imread(img_path)
    if img.shape == sample_img_shape:    
        all_x[sample_i] = img
        all_y[sample_i] = np.array([1,0])
        sample_i += 1

print("Collected %d samples!" % num_samples)

Image shape: 64 x 64 x 3
Collected 3000 samples!


In [160]:
# Create dataloader
train_dataset = []
for i in range(len(all_x)):
    img = cv2.cvtColor(all_x[i], cv2.COLOR_BGR2GRAY)
    img = cv2.normalize(img, img, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    train_dataset.append({'predicted': img, 'ground_truth': all_y[i]})
    
dataloader = DataLoader(train_dataset, batch_size=4,
                        shuffle=True, num_workers=8)

In [161]:
#plt.imshow(train_dataset[1]['predicted'], cmap='gray')
#plt.show()

In [162]:
max_epoch = 3
# max_epoch = 100

# Just a progress bar
# If it doesn't load (js error), run this in terminal then restart jupyter:
# jupyter nbextension enable --py --sys-prefix widgetsnbextension
f = FloatProgress(min=0, max=max_epoch*len(dataloader))
display(f)
    
for epoch in range(max_epoch):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(dataloader):
        # get the inputs
        inputs, labels = data['predicted'], data['ground_truth']

       # wrap them in Variable
        inputs, targets = Variable(inputs.float()).view(-1,1,64,64), \
                         Variable(labels.float()).view(-1, 2)

       # zero the parameter gradients
        optimizer.zero_grad()

       # forward + backward + optimize
        outputs = vgg(inputs)

        print("----------------------\nOutputs:")
        print(outputs)
        print("\nTargets:")
        print(targets)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
            
       # print statistics
        running_loss += loss.data[0]
        print(running_loss)
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        f.value += 1
f.close()
print('Finished Training')

----------------------
Outputs:
Variable containing:
 0.5271  0.4729
 0.4464  0.5536
 0.4720  0.5280
 0.5057  0.4943
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 1  0
 1  0
[torch.FloatTensor of size 4x2]

0.666036963463
----------------------
Outputs:
Variable containing:
 0.6670  0.3330
 0.6252  0.3748
 0.4885  0.5115
 0.5134  0.4866
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 1  0
 1  0
[torch.FloatTensor of size 4x2]

1.35832434893
----------------------
Outputs:
Variable containing:
 0.6417  0.3583
 0.5287  0.4713
 0.6025  0.3975
 0.5272  0.4728
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 1  0
 1  0
 1  0
[torch.FloatTensor of size 4x2]

1.91525608301
----------------------
Outputs:
Variable containing:
 0.5648  0.4352
 0.6903  0.3097
 0.5959  0.4041
 0.6010  0.3990
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 0  1
 1  0
 1  0
[torch.FloatTensor of size 4x2]

2.673

22.6635774374
----------------------
Outputs:
Variable containing:
 0.4657  0.5343
 0.2648  0.7352
 0.1808  0.8192
 0.2529  0.7471
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 0  1
 1  0
[torch.FloatTensor of size 4x2]

23.3251283765
----------------------
Outputs:
Variable containing:
 0.1273  0.8727
 0.4940  0.5060
 0.1749  0.8251
 0.4083  0.5917
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 1  0
 0  1
 0  1
[torch.FloatTensor of size 4x2]

23.7147366405
----------------------
Outputs:
Variable containing:
 0.4001  0.5999
 0.3424  0.6576
 0.4614  0.5386
 0.2250  0.7750
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 1  0
 0  1
[torch.FloatTensor of size 4x2]

24.3056923151
----------------------
Outputs:
Variable containing:
 0.2516  0.7484
 0.4499  0.5501
 0.3809  0.6191
 0.1170  0.8830
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 1  0
 1  0
 0  1
[torch.FloatTensor of siz

43.4852759745
----------------------
Outputs:
Variable containing:
 0.9517  0.0483
 0.5585  0.4415
 0.2058  0.7942
 0.8269  0.1731
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 0  1
 0  1
[torch.FloatTensor of size 4x2]

44.1980941799
----------------------
Outputs:
Variable containing:
 0.0972  0.9028
 0.1237  0.8763
 0.9809  0.0191
 0.9951  0.0049
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 0  1
 1  0
 1  0
[torch.FloatTensor of size 4x2]

44.2627279665
----------------------
Outputs:
Variable containing:
 0.2766  0.7234
 0.6153  0.3847
 0.8279  0.1721
 0.2075  0.7925
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 0  1
 0  1
 0  1
[torch.FloatTensor of size 4x2]

45.0806042459
----------------------
Outputs:
Variable containing:
 0.8773  0.1227
 0.9646  0.0354
 0.0390  0.9610
 0.8541  0.1459
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 0  1
 0  1
[torch.FloatTensor of siz

Targets:
Variable containing:
 1  0
 0  1
 0  1
 1  0
[torch.FloatTensor of size 4x2]

65.2723960243
----------------------
Outputs:
Variable containing:
 0.0640  0.9360
 0.2285  0.7715
 0.1576  0.8424
 0.0275  0.9725
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 0  1
 0  1
 0  1
[torch.FloatTensor of size 4x2]

65.4036515169
----------------------
Outputs:
Variable containing:
 0.1949  0.8051
 0.9848  0.0152
 0.9768  0.0232
 0.9763  0.0237
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 1  0
 0  1
 1  0
[torch.FloatTensor of size 4x2]

66.7632632665
----------------------
Outputs:
Variable containing:
 0.9986  0.0014
 0.9985  0.0015
 0.9897  0.0103
 0.2804  0.7196
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 1  0
 0  1
 0  1
[torch.FloatTensor of size 4x2]

67.9899602346
----------------------
Outputs:
Variable containing:
 0.2107  0.7893
 0.3416  0.6584
 0.3227  0.6773
 0.9957  0.0043
[torch.FloatTensor of size


82.3157541044
----------------------
Outputs:
Variable containing:
 0.9187  0.0813
 0.9845  0.0155
 0.9160  0.0840
 0.8649  0.1351
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 1  0
 1  0
 1  0
[torch.FloatTensor of size 4x2]

82.3990669064
----------------------
Outputs:
Variable containing:
 0.0014  0.9986
 0.9484  0.0516
 0.0729  0.9271
 0.9835  0.0165
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 1  0
 0  1
 1  0
[torch.FloatTensor of size 4x2]

82.4357547984
----------------------
Outputs:
Variable containing:
 0.5295  0.4705
 0.9855  0.0145
 0.9654  0.0346
 0.2198  0.7802
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 1  0
 0  1
 1  0
[torch.FloatTensor of size 4x2]

83.8474853262
----------------------
Outputs:
Variable containing:
 0.9783  0.0217
 0.0028  0.9972
 0.9658  0.0342
 0.7008  0.2992
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 1  0
 1  0
[torch.FloatTensor of si

...

42.3303510052
----------------------
Outputs:
Variable containing:
 9.5812e-01  4.1882e-02
 6.5022e-08  1.0000e+00
 9.9897e-01  1.0346e-03
 9.9684e-01  3.1644e-03
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 1  0
 1  0
[torch.FloatTensor of size 4x2]

42.3420981465
----------------------
Outputs:
Variable containing:
 0.0001  0.9999
 0.9993  0.0007
 0.9924  0.0076
 0.0093  0.9907
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 1  0
 1  0
 0  1
[torch.FloatTensor of size 4x2]

42.3465662397
----------------------
Outputs:
Variable containing:
 9.9947e-01  5.3445e-04
 1.7378e-07  1.0000e+00
 9.9633e-01  3.6696e-03
 9.8334e-01  1.6663e-02
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 1  0
 1  0
[torch.FloatTensor of size 4x2]

42.3518199323
----------------------
Outputs:
Variable containing:
 9.6997e-07  1.0000e+00
 7.0937e-03  9.9291e-01
 9.9707e-01  2.9271e-03
 9.9988e-01  1.1644e-04
[torch.FloatTenso

44.3477356748
----------------------
Outputs:
Variable containing:
 2.2449e-06  1.0000e+00
 9.9904e-01  9.5937e-04
 6.9053e-03  9.9309e-01
 9.9540e-01  4.6029e-03
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 1  0
 1  0
 1  0
[torch.FloatTensor of size 4x2]

45.5929975824
----------------------
Outputs:
Variable containing:
 3.7873e-04  9.9962e-01
 9.9645e-01  3.5506e-03
 2.7029e-07  1.0000e+00
 9.9533e-01  4.6684e-03
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 1  0
 0  1
 1  0
[torch.FloatTensor of size 4x2]

45.5951514158
----------------------
Outputs:
Variable containing:
 0.9844  0.0156
 0.9933  0.0067
 0.9987  0.0013
 0.0000  1.0000
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 1  0
 1  0
 0  1
[torch.FloatTensor of size 4x2]

45.6010990154
----------------------
Outputs:
Variable containing:
 6.5192e-07  1.0000e+00
 3.0652e-05  9.9997e-01
 9.9846e-01  1.5385e-03
 7.1397e-04  9.9929e-01
[torch.FloatTenso

46.0442569687
----------------------
Outputs:
Variable containing:
 0.9995  0.0005
 0.0000  1.0000
 0.0002  0.9998
 0.0008  0.9992
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 0  1
 0  1
[torch.FloatTensor of size 4x2]

46.044646219
----------------------
Outputs:
Variable containing:
 0.0195  0.9805
 0.0001  0.9999
 0.9995  0.0005
 0.0001  0.9999
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 0  1
 1  0
 0  1
[torch.FloatTensor of size 4x2]

46.0497357278
----------------------
Outputs:
Variable containing:
 0.0013  0.9987
 0.0000  1.0000
 0.9998  0.0002
 0.9769  0.0231
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 0  1
 1  0
 1  0
[torch.FloatTensor of size 4x2]

46.0559768552
----------------------
Outputs:
Variable containing:
 0.9998  0.0002
 0.9885  0.0115
 0.0079  0.9921
 0.0002  0.9998
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 1  0
 0  1
 0  1
[torch.FloatTensor of size


47.4951035961
----------------------
Outputs:
Variable containing:
 9.9982e-01  1.7679e-04
 9.8403e-01  1.5975e-02
 2.0941e-06  1.0000e+00
 1.2617e-05  9.9999e-01
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 1  0
 0  1
 0  1
[torch.FloatTensor of size 4x2]

47.4991774684
----------------------
Outputs:
Variable containing:
 0.0002  0.9998
 0.0019  0.9981
 0.0000  1.0000
 0.9994  0.0006
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 0  1
 0  1
 0  1
 1  0
[torch.FloatTensor of size 4x2]

47.4998681849
----------------------
Outputs:
Variable containing:
 9.9593e-01  4.0702e-03
 8.8065e-06  9.9999e-01
 2.3770e-02  9.7623e-01
 1.6945e-03  9.9831e-01
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 0  1
 0  1
[torch.FloatTensor of size 4x2]

47.507328122
----------------------
Outputs:
Variable containing:
 0.9750  0.0250
 0.0020  0.9980
 0.0000  1.0000
 0.9995  0.0005
[torch.FloatTensor of size 4x2]


Targets:
Variab

48.1733927254
----------------------
Outputs:
Variable containing:
 9.9759e-01  2.4148e-03
 2.2024e-04  9.9978e-01
 8.2750e-06  9.9999e-01
 6.8215e-05  9.9993e-01
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 0  1
 0  1
 0  1
[torch.FloatTensor of size 4x2]

48.1740713344
----------------------
Outputs:
Variable containing:
 0.9998  0.0002
 0.9979  0.0021
 0.0003  0.9997
 0.0045  0.9955
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 1  0
 0  1
 0  1
[torch.FloatTensor of size 4x2]

48.1758330583
----------------------
Outputs:
Variable containing:
 1.0000  0.0000
 0.9999  0.0001
 0.9990  0.0010
 0.9999  0.0001
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 1  0
 1  0
 1  0
[torch.FloatTensor of size 4x2]

48.1761392121
----------------------
Outputs:
Variable containing:
 0.9922  0.0078
 0.9992  0.0008
 1.0000  0.0000
 0.9999  0.0001
[torch.FloatTensor of size 4x2]


Targets:
Variable containing:
 1  0
 1  0
 1  0

In [3]:
# Load Testing Data
THRESH = 0.5
TRIAL = "36"
TRIAL_DIR = "/Users/evan/workspace/spec/mm_backend/resources/trials/" + TRIAL

test_dataset = []
results = {0: [], 1:[], -1: []}
raw_paths = os.listdir(TRIAL_DIR)
for path in raw_paths:
    if not path.startswith("full_face") and path.endswith(".jpg"):
        img = cv2.imread(TRIAL_DIR + "/" + path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.normalize(img, img, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        # Test each image
        test_dataset.append({'predicted': img, 'ground_truth': np.array([0, 0]).T})


dataloader = DataLoader(test_dataset, batch_size=1,
                        shuffle=True, num_workers=8)

for i, data in enumerate(dataloader):
    # get the inputs
    inputs, labels = data['predicted'], data['ground_truth']
        
    # wrap them in Variable
    inputs, targets = Variable(inputs.float()).view(-1,1,64,64), \
        Variable(labels.float()).view(-1, 2)

    # forward + backward + optimize
    test_outputs = vgg(inputs)

    print test_outputs

with open("vgg_filter_results.json",'w') as f:
    json.dump(results, f)

pprint(results)
print("\nCOUNTS\n0:%d\n1:%d,E:%d" % (len(results[0]), len(results[1]), len(results[-1])))

for path in results[1]:
    cv2.imshow('mole',cv2.imread(TRIAL_DIR + "/" + path))
    cv2.waitKey(0)

NameError: name 'vgg' is not defined